<a href="https://colab.research.google.com/github/arinafil/informatic_ds/blob/main/Lucky_Number_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import scipy.stats as sps

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from functools import lru_cache
from Levenshtein import distance as levenshtein_distance
from collections import defaultdict
import numpy as np

# Прочтение таблиц

In [3]:
df = pd.read_excel('/content/Lucky_Number_5.xlsx')
df.head()

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene
0,0,18801,0.021561,TGTGCCAGCAGTTTATACGGGGACCAAGAGACCCAGTACTTC,CASSLYGDQETQYF,TRBV27,TRBJ2-5
1,1,8629,0.009896,TGTGCCAGCAGATCTGGGGCCAACGTCCTGACTTTC,CASRSGANVLTF,TRBV4-2,TRBJ2-6
2,2,7952,0.009120,TGTGCCAGCAGTTTTCGGACAGGGCCCCCGCCCTACAATGAGCAGT...,CASSFRTGPPPYNEQFF,TRBV6-2,TRBJ2-1
3,3,5258,0.006030,TGCGCCAGCAGCCAAGATCCAGGGGGGCCCTCTGGGGCCAACGTCC...,CASSQDPGGPSGANVLTF,TRBV4-3,TRBJ2-6
4,4,4782,0.005484,TGTGCCAGCAGCTTGCAGGGAGATACGCAGTATTTT,CASSLQGDTQYF,TRBV11-3,TRBJ2-3


In [4]:
database = pd.read_excel('/content/vdjdb_slim.xlsx')

In [5]:
database_exp = database.rename(columns={'cdr3': 'CDR3.amino.acid.sequence'})

In [6]:
data_claster = df[:10000]
data_claster

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene
0,0,18801,0.021561,TGTGCCAGCAGTTTATACGGGGACCAAGAGACCCAGTACTTC,CASSLYGDQETQYF,TRBV27,TRBJ2-5
1,1,8629,0.009896,TGTGCCAGCAGATCTGGGGCCAACGTCCTGACTTTC,CASRSGANVLTF,TRBV4-2,TRBJ2-6
2,2,7952,0.009120,TGTGCCAGCAGTTTTCGGACAGGGCCCCCGCCCTACAATGAGCAGT...,CASSFRTGPPPYNEQFF,TRBV6-2,TRBJ2-1
3,3,5258,0.006030,TGCGCCAGCAGCCAAGATCCAGGGGGGCCCTCTGGGGCCAACGTCC...,CASSQDPGGPSGANVLTF,TRBV4-3,TRBJ2-6
4,4,4782,0.005484,TGTGCCAGCAGCTTGCAGGGAGATACGCAGTATTTT,CASSLQGDTQYF,TRBV11-3,TRBJ2-3
...,...,...,...,...,...,...,...
9995,10471,5,0.000006,TGTGCCAGCACCATGGACCGCGCTGCCTATGGCTACACCTTC,CASTMDRAAYGYTF,TRBV12-4,TRBJ1-2
9996,9400,5,0.000006,TGTGCCAGCACCCAACGGGGGGGATCTCCCTACAATGAGCAGTTCTTC,CASTQRGGSPYNEQFF,TRBV12-4,TRBJ2-1
9997,9810,5,0.000006,TGTGCCAGCACCCACTCGGACAGGGCCAACACTGAAGCTTTCTTT,CASTHSDRANTEAFF,TRBV12-3,TRBJ1-1
9998,11484,5,0.000006,TGTGCCAGCACCCCAGGGGGCCGGCCCCAGCATTTT,CASTPGGRPQHF,TRBV12-3,TRBJ1-5


# Функция кластеризации

In [7]:
# Кэшированная функция для вычисления расстояния Левенштейна
@lru_cache(maxsize=None)
def cached_levenshtein_distance(seq1, seq2):
    return levenshtein_distance(seq1, seq2)

# Функция для кластеризации последовательностей
def cluster_sequences(sequences, max_distance=1):
    clusters = []
    for seq in sequences:
        added_to_cluster = False
        for cluster in clusters:
            if any(cached_levenshtein_distance(seq, existing_seq) <= max_distance for existing_seq in cluster):
                cluster.append(seq)
                added_to_cluster = True
                break
        if not added_to_cluster:
            clusters.append([seq])
    return clusters

# Функция для объединения кластеров
def merge_clusters(clusters, max_distance=1):
    merged_clusters = []
    for cluster in clusters:
        added_to_cluster = False
        for merged_cluster in merged_clusters:
            if any(cached_levenshtein_distance(seq, existing_seq) <= max_distance for seq in cluster for existing_seq in merged_cluster):
                merged_cluster.extend(cluster)
                added_to_cluster = True
                break
        if not added_to_cluster:
            merged_clusters.append(cluster)
    return merged_clusters

# Пример последовательностей аминокислот
sequences = data_claster['CDR3.amino.acid.sequence']

# Разделение данных на части
chunk_size = len(sequences) // 100
chunks = [sequences[i:i + chunk_size] for i in range(0, len(sequences), chunk_size)]

# Кластеризация каждой части
all_clusters = []
for chunk in chunks:
    all_clusters.extend(cluster_sequences(chunk, max_distance=1))

# Объединение кластеров
final_clusters = merge_clusters(all_clusters, max_distance=1)

# Вывод результатов
# Создание DataFrame
cluster_data = []
for i, cluster in enumerate(final_clusters):
    for seq in cluster:
        cluster_data.append({'Cluster': i + 1, 'Sequence': seq})

results_df = pd.DataFrame(cluster_data)

In [8]:
# Создание нового DataFrame с данными из data_claster и номером кластера
cluster_assignments = {}
for i, cluster in enumerate(final_clusters):
  for seq in cluster:
    cluster_assignments[seq] = i + 1

results_df = data_claster.copy()
results_df['Cluster'] = results_df['CDR3.amino.acid.sequence'].map(cluster_assignments)

# Вывод нового DataFrame results_df
results_df

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene,Cluster
0,0,18801,0.021561,TGTGCCAGCAGTTTATACGGGGACCAAGAGACCCAGTACTTC,CASSLYGDQETQYF,TRBV27,TRBJ2-5,1
1,1,8629,0.009896,TGTGCCAGCAGATCTGGGGCCAACGTCCTGACTTTC,CASRSGANVLTF,TRBV4-2,TRBJ2-6,2
2,2,7952,0.009120,TGTGCCAGCAGTTTTCGGACAGGGCCCCCGCCCTACAATGAGCAGT...,CASSFRTGPPPYNEQFF,TRBV6-2,TRBJ2-1,3
3,3,5258,0.006030,TGCGCCAGCAGCCAAGATCCAGGGGGGCCCTCTGGGGCCAACGTCC...,CASSQDPGGPSGANVLTF,TRBV4-3,TRBJ2-6,4
4,4,4782,0.005484,TGTGCCAGCAGCTTGCAGGGAGATACGCAGTATTTT,CASSLQGDTQYF,TRBV11-3,TRBJ2-3,5
...,...,...,...,...,...,...,...,...
9995,10471,5,0.000006,TGTGCCAGCACCATGGACCGCGCTGCCTATGGCTACACCTTC,CASTMDRAAYGYTF,TRBV12-4,TRBJ1-2,8928
9996,9400,5,0.000006,TGTGCCAGCACCCAACGGGGGGGATCTCCCTACAATGAGCAGTTCTTC,CASTQRGGSPYNEQFF,TRBV12-4,TRBJ2-1,8929
9997,9810,5,0.000006,TGTGCCAGCACCCACTCGGACAGGGCCAACACTGAAGCTTTCTTT,CASTHSDRANTEAFF,TRBV12-3,TRBJ1-1,8930
9998,11484,5,0.000006,TGTGCCAGCACCCCAGGGGGCCGGCCCCAGCATTTT,CASTPGGRPQHF,TRBV12-3,TRBJ1-5,8931


# Подсчёт Total Read Proportion

In [9]:
# Подсчет количества последовательностей в каждом кластере
cluster_counts = {i + 1: len(cluster) for i, cluster in enumerate(final_clusters)}
# Подсчет суммарного Read.proportion для каждого кластера
cluster_proportions = {}
for index, row in data_claster.iterrows():
  seq = row['CDR3.amino.acid.sequence']
  proportion = row['Read.proportion']
  for i, cluster in enumerate(final_clusters):
    if seq in cluster:
      cluster_proportions[i + 1] = cluster_proportions.get(i + 1, 0) + proportion
      break

# Создание DataFrame с суммарными пропорциями, номерами кластеров и количеством последовательностей
proportions_data = []
for cluster_number, total_proportion in cluster_proportions.items():
  proportions_data.append({'Cluster': cluster_number,
               'Total Read Proportion': total_proportion,
               'Sequence Count': cluster_counts[cluster_number]})


# Сортировка по Sequence Count

In [10]:
proportions_df = pd.DataFrame(proportions_data)
proportions_df = proportions_df.sort_values(by='Sequence Count', ascending=False)
proportions_df

,Cluster,Total Read Proportion,Sequence Count
4,5,0.006188,66
13,14,0.003157,23
36,37,0.001299,17
795,700,0.000225,17
184,185,0.000378,16
...,...,...,...
3165,3172,0.000014,1
3164,3171,0.000014,1
3163,3170,0.000014,1
3162,3169,0.000014,1


# Сортировка по Total Read Proportion

In [11]:
proportions_df_sorted = proportions_df.sort_values(by='Total Read Proportion', ascending=False)
proportions_df_sorted = proportions_df_sorted.loc[proportions_df_sorted['Sequence Count'] > 1]
proportions_df_sorted

,Cluster,Total Read Proportion,Sequence Count
1,2,0.009903,2
4,5,0.006188,66
6,7,0.004693,4
17,18,0.003858,3
13,14,0.003157,23
...,...,...,...
8867,8871,0.000011,2
8656,8660,0.000011,2
8262,8266,0.000011,2
8456,8460,0.000011,2


# Выбор кластера

In [12]:
full = pd.merge(results_df, database_exp , on='CDR3.amino.acid.sequence', how='inner')

In [13]:
full = full[['Cluster', 'CDR3.amino.acid.sequence', 'Read.proportion', 'bestVGene',	'bestJGene', 'v.segm',	'j.segm', 'mhc.class', 'species', 'antigen.species', 'antigen.gene']]
full

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
0,37,CASSSGTGAYEQYF,0.001098,TRBV7-2,TRBJ2-7,TRBV7-9*01,TRBJ2-7*01,MHCI,HomoSapiens,CMV,IE1
1,49,CASSPGQNEQFF,0.000739,TRBV9,TRBJ2-1,TRBV18*01,TRBJ2-1*01,MHCII,HomoSapiens,InfluenzaA,HA
2,179,CASSPRTSGADTQYF,0.000159,TRBV12-4,TRBJ2-3,TRBV2*01,TRBJ2-3*01,MHCII,HomoSapiens,HIV-1,Gag
3,50,CASSYGGDGYTF,0.000158,TRBV6-5,TRBJ1-2,TRBV6-5*01,TRBJ1-2*01,MHCI,HomoSapiens,CMV,IE1
4,201,CASSLGPNTEAFF,0.000135,TRBV12-4,TRBJ1-1,"TRBV12-3*01,TRBV7-2*01",TRBJ1-1*01,MHCI,HomoSapiens,CMV,IE1
...,...,...,...,...,...,...,...,...,...,...,...
412,1884,CASSLAGGYEQYF,0.000006,TRBV5-1,TRBJ2-7,TRBV28*01,TRBJ2-7*01,MHCI,HomoSapiens,SARS-CoV-2,ORF1ab
413,1884,CASSLAGGYEQYF,0.000006,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCI,HomoSapiens,CMV,pp65
414,5,CASSLGTDTQYF,0.000006,TRBV5-1,TRBJ2-3,TRBV5-1*01,TRBJ2-3*01,MHCI,HomoSapiens,HomoSapiens,SF3B1
415,5,CASSLGTDTQYF,0.000006,TRBV5-1,TRBJ2-3,TRBV7-9*01,TRBJ2-3*01,MHCI,HomoSapiens,CMV,IE1


In [14]:
full_vj = full.loc[(full['bestVGene'].str[:7] == full['v.segm'].str[:7]) & (full['bestJGene'].str[:7] == full['j.segm'].str[:7])]
full_vj

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
3,50,CASSYGGDGYTF,0.000158,TRBV6-5,TRBJ1-2,TRBV6-5*01,TRBJ1-2*01,MHCI,HomoSapiens,CMV,IE1
4,201,CASSLGPNTEAFF,0.000135,TRBV12-4,TRBJ1-1,"TRBV12-3*01,TRBV7-2*01",TRBJ1-1*01,MHCI,HomoSapiens,CMV,IE1
5,201,CASSLGPNTEAFF,0.000135,TRBV12-4,TRBJ1-1,TRBV12-3*01,TRBJ1-1*01,MHCI,HomoSapiens,EBV,EBNA4
12,465,CASSLGQGDGYTF,0.000065,TRBV7-9,TRBJ1-2,TRBV7-9*01,TRBJ1-2*01,MHCI,HomoSapiens,HomoSapiens,MLANA
20,624,CASSQGGYGYTF,0.000052,TRBV4-3,TRBJ1-2,TRBV4-3*01,TRBJ1-2*01,MHCI,HomoSapiens,YFV,NS4B
...,...,...,...,...,...,...,...,...,...,...,...
401,6816,CSARENTGELFF,0.000006,TRBV20-1,TRBJ2-2,TRBV20-1*01,TRBJ2-2*01,MHCI,HomoSapiens,EBV,EBNA3A
402,2209,CSARAGDTEAFF,0.000006,TRBV20-1,TRBJ1-1,TRBV20-1*01,TRBJ1-1*01,MHCI,HomoSapiens,CMV,IE1
409,8727,CASSSDGMNTEAFF,0.000006,TRBV10-2,TRBJ1-1,TRBV10-2*01,TRBJ1-1*01,MHCI,HomoSapiens,EBV,LMP2A
413,1884,CASSLAGGYEQYF,0.000006,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCI,HomoSapiens,CMV,pp65


In [16]:
full_vj.value_counts()

Cluster  CDR3.amino.acid.sequence  Read.proportion  bestVGene  bestJGene  v.segm       j.segm      mhc.class  species      antigen.species  antigen.gene
4288     CSASGDRGADTQYF            0.000008         TRBV20-1   TRBJ2-3    TRBV20-1*01  TRBJ2-3*01  MHCI       HomoSapiens  YFV              NS4B            2
6224     CASSLNPVNTEAFF            0.000008         TRBV7-3    TRBJ1-1    TRBV7-3*01   TRBJ1-1*01  MHCI       HomoSapiens  EBV              EBNA4           2
2313     CASSKQGGGYGYTF            0.000018         TRBV6-5    TRBJ1-2    TRBV6-5*01   TRBJ1-2*01  MHCI       HomoSapiens  EBV              LMP2A           1
3134     CSAPTSGGSNEQFF            0.000014         TRBV20-1   TRBJ2-1    TRBV20-1*01  TRBJ2-1*01  MHCI       HomoSapiens  YFV              NS4B            1
                                   0.000009         TRBV20-1   TRBJ2-1    TRBV20-1*01  TRBJ2-1*01  MHCI       HomoSapiens  YFV              NS4B            1
                                                                                                                                                           ..
624      CASSQGGYGYTF              0.000007         TRBV4-3    TRBJ1-2    TRBV4-3*01   TRBJ1-2*01  MHCI       HomoSapiens  YFV              NS4B            1
465      CASSLGQGDGYTF             0.000065         TRBV7-9    TRBJ1-2    TRBV7-9*01   TRBJ1-2*01  MHCI       HomoSapiens  HomoSapiens      MLANA           1
425      CASSLTSGPYNEQFF           0.000014         TRBV7-2    TRBJ2-1    TRBV7-2*01   TRBJ2-1*01  MHCI       HomoSapiens  CMV              IE1             1
270      CASSLYTDTQYF              0.000017         TRBV7-2    TRBJ2-3    TRBV7-2*01   TRBJ2-3*01  MHCI       HomoSapiens  HomoSapiens      WT1             1
8727     CASSSDGMNTEAFF            0.000006         TRBV10-2   TRBJ1-1    TRBV10-2*01  TRBJ1-1*01  MHCI       HomoSapiens  EBV              LMP2A           1
Name: count, Length: 72, dtype: int64

# Обработка для одного выбранного кластера

In [17]:
results_df_one_cluster = results_df.loc[results_df['Cluster'] == 4288]
results_df_one_cluster = results_df_one_cluster.rename(columns={'Sequence': 'CDR3.amino.acid.sequence'})
results_df_one_cluster

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene,Cluster
4606,5128,9,0.000010,TGCAGTGCGGGGGACAGGGGCGCAGATACGCAGTATTTT,CSAGDRGADTQYF,TRBV20-1,TRBJ2-3,4288
6320,6933,7,0.000008,TGCAGTGCTAGCGGGGACAGGGGGGCAGATACGCAGTATTTT,CSASGDRGADTQYF,TRBV20-1,TRBJ2-3,4288
6336,6931,7,0.000008,TGCAGTGCTAGTGGGGACAGGGGAGCAGATACGCAGTATTTT,CSASGDRGADTQYF,TRBV20-1,TRBJ2-3,4288


In [18]:
full = pd.merge(results_df_one_cluster, database_exp , on='CDR3.amino.acid.sequence', how='inner')

In [19]:
full = full[['Cluster', 'CDR3.amino.acid.sequence', 'Read.proportion', 'bestVGene',	'bestJGene', 'v.segm',	'j.segm', 'mhc.class', 'species', 'antigen.species', 'antigen.gene']]
full

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
0,4288,CSASGDRGADTQYF,0.000008,TRBV20-1,TRBJ2-3,TRBV20-1*01,TRBJ2-3*01,MHCI,HomoSapiens,YFV,NS4B
1,4288,CSASGDRGADTQYF,0.000008,TRBV20-1,TRBJ2-3,TRBV20-1*01,TRBJ2-3*01,MHCI,HomoSapiens,YFV,NS4B


# Разборки с V и J

In [20]:
full_vj = full.loc[(full['bestVGene'].str[:7] == full['v.segm'].str[:7]) & (full['bestJGene'].str[:7] == full['j.segm'].str[:7])]
full_vj

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
0,4288,CSASGDRGADTQYF,0.000008,TRBV20-1,TRBJ2-3,TRBV20-1*01,TRBJ2-3*01,MHCI,HomoSapiens,YFV,NS4B
1,4288,CSASGDRGADTQYF,0.000008,TRBV20-1,TRBJ2-3,TRBV20-1*01,TRBJ2-3*01,MHCI,HomoSapiens,YFV,NS4B
